In [1]:
import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm.auto import tqdm
import cv2

# from tensorflow.keras.applications import Densnet
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
import tensorflow.keras as keras
from tensorflow.data import Dataset

from tensorflow.keras.optimizers import Adam, RMSprop, Nadam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from itertools import product
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skmultilearn.model_selection import iterative_train_test_split
from wandb.keras import WandbCallback
import wandb

2022-06-18 22:04:00.090322: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
tf.__version__

'2.4.1'

In [4]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'pretrain_net': {
            'values': ['inception', 'resnet']
        },
        'batch_size': {
            'values': [20, 50]
        },
        'dropout': {
            'values': [0.2, 0.5]
        },
        'dense': {
            'values': [32, 256, 1024]
        },
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4, 3e-5]
        },
        'optimizer': {
            'values': ['adam', 'sgd', 'rmsprop']
        },
        'activation': {
            'values': ['relu', 'elu', 'selu', 'softmax']
        }
    }
}

In [5]:
IMAGE_SIZE = 320

In [6]:
df = pd.read_csv('/home/lab38/Multi_proj_6/data/straw_smallsizeimg_with_pest.csv')
df.head()

,image,grow,disease,area,points,original,disease-grow
0,/home/lab38/딸기/Strawberry Pest Damage_3.jpg,5,11,NaN,NaN,NaN,11-5
1,/home/lab38/딸기/Strawberry Pest Damage_727.jpg,2,11,NaN,NaN,NaN,11-2
2,/home/lab38/딸기/Strawberry Pest Damage_487.jpg,5,11,NaN,NaN,NaN,11-5
3,/home/lab38/딸기/Strawberry Pest Damage_124.jpg,5,11,NaN,NaN,NaN,11-5
4,/home/lab38/딸기/Strawberry Pest Damage_70.jpg,5,11,NaN,NaN,NaN,11-5


In [7]:
# label encoding
disease_encoder = LabelEncoder()
disease_encoder.fit(df['disease'])
df['disease'] = disease_encoder.transform(df['disease'])
print(df['disease'].unique())

grow_encoder = LabelEncoder()
grow_encoder.fit(df['grow'])
df['grow'] = grow_encoder.transform(df['grow'])
print(df['grow'].unique())

[3 0 1 2]
[4 1 3 0 2]


In [8]:
shuffled_df = df.sample(frac=1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df['image'],
                                                   df['disease-grow'],
                                                   stratify=df['disease-grow'],
                                                   test_size=0.2)

In [10]:
train_df = df[df['image'].isin(X_train)]
test_df = df[df['image'].isin(X_test)]

In [11]:
print(train_df['disease'].unique())
print(train_df['grow'].unique())
print(test_df['disease'].unique())
print(test_df['grow'].unique())

[3 0 1 2]
[4 1 3 0 2]
[3 0 1 2]
[1 4 3 0 2]


In [12]:
train_gen = ImageDataGenerator(rescale=1./255,
                            rotation_range=20, # 최대 20도까지 회전
                            width_shift_range=0.1, # 최대 x 범위안에서 좌우/상하 이동
                            height_shift_range=0.1,
                            zoom_range=0.2, # 확대 축소 비율,
                            horizontal_flip=True, # 좌우반전
                            vertical_flip=True, # 상하반전
                            fill_mode='nearest')
valid_gen = ImageDataGenerator(rescale= 1. /255.)

In [13]:
def train() :
    wandb.init()
    tf.keras.backend.clear_session()
    config_defaults = {
        'pretrain_net': 'inception',
        'epochs' : 10,
        'batch_size': 50,
        'dropout' : 0.5,
        'learning_rate' : 1e-3,
        'activation': 'relu',
        'optimizer': 'adam'
    }
    
    wandb.init(config=config_defaults)
    config = wandb.config
    
    train_generator = train_gen.flow_from_dataframe(train_df, 
                                               x_col='image',
                                               y_col=['disease', 'grow'],
                                               target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                               class_mode='multi_output',
                                               batch_size=config.batch_size)
    valid_generator = valid_gen.flow_from_dataframe(test_df,
                                               x_col='image',
                                               y_col=['disease','grow'],
                                               target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                               class_mode='multi_output',
                                               batch_size=config.batch_size)
    if config.pretrain_net == 'inception' :
        MODEL_IMAGE_SIZE = 299
        base_model = inception_resnet_v2.InceptionResNetV2(
            weights='imagenet',
            include_top = False,
            input_shape = (MODEL_IMAGE_SIZE, MODEL_IMAGE_SIZE,3)
        )
    elif config.pretrain_net == 'resnet' :
        MODEL_IMAGE_SIZE = 224
        base_model= resnet50.ResNet50(
              weights='imagenet',
              include_top=False,
              input_shape = (MODEL_IMAGE_SIZE, MODEL_IMAGE_SIZE,3)
             )
    base_model.trainable = False 

    
    # resizing model
    input_data = layers.Input((IMAGE_SIZE, IMAGE_SIZE, 3))
    x = tf.keras.layers.experimental.preprocessing.Resizing(MODEL_IMAGE_SIZE, MODEL_IMAGE_SIZE)(input_data)
    resizing = Model(inputs=input_data, outputs=x, name='resize')
   
    # model
    inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    x = resizing(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(config.dropout)(x)
    backbone_out = layers.Dense(config.dense, activation=config.activation)(x)

    disease_outputs = layers.Dense(df['disease'].nunique(), activation='softmax',
                            name = 'diease_outputs')(backbone_out)
    grow_outputs = layers.Dense(df['grow'].nunique(), activation='softmax',
                        name = 'grow_outputs')(backbone_out)

    model = Model(inputs=inputs, 
                  outputs=[disease_outputs, grow_outputs],
                  name='strawberry')    
    if config.optimizer=='sgd':
        optimizer = SGD(learning_rate=config.learning_rate)
    elif config.optimizer=='rmsprop':
        optimizer = RMSprop(learning_rate=config.learning_rate)
    elif config.optimizer=='adam':
        optimizer = Adam(learning_rate=config.learning_rate)

    model.compile(loss={
                  'diease_outputs' : 'sparse_categorical_crossentropy',
                  'grow_outputs' : 'sparse_categorical_crossentropy'
                  },
                  optimizer=optimizer,
                  metrics=['accuracy'])    
    
    _ = model.fit(train_generator,
          validation_data=valid_generator,
          verbose=1,
          epochs=config.epochs,
          callbacks=[WandbCallback()],
          steps_per_epoch=len(train_df)//config.batch_size)


In [14]:
sweep_id = wandb.sweep(sweep_config, project='strawberry')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: y71b563r
Sweep URL: https://wandb.ai/hkleee/strawberry/sweeps/y71b563r


In [ ]:
#wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 0cdnlkdd with config:
wandb: 	activation: softmax
wandb: 	batch_size: 50
wandb: 	dense: 256
wandb: 	dropout: 0.5
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: rmsprop
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hkleee. Use `wandb login --relogin` to force relogin


Found 13437 validated image filenames.
Found 3360 validated image filenames.


2022-06-17 13:25:07.158490: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-17 13:25:07.159642: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-17 13:25:07.203184: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-17 13:25:07.203841: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-06-17 13:25:07.203895: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-17 13:25:07.283430: I tensorflow/stream_executor/platform/default/dso_loade

Epoch 1/10


2022-06-17 13:25:24.922408: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-06-17 13:25:25.980256: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


...

wandb: Agent Starting Run: r0t8izdi with config:
wandb: 	activation: relu
wandb: 	batch_size: 50
wandb: 	dense: 256
wandb: 	dropout: 0.2
wandb: 	learning_rate: 0.001
wandb: 	optimizer: sgd
wandb: 	pretrain_net: resnet
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13437 validated image filenames.
Found 3360 validated image filenames.
Epoch 1/10
268/268 [==============================] - 303s 1s/step - loss: 2.8280 - diease_outputs_loss: 1.2212 - grow_outputs_loss: 1.6069 - diease_outputs_accuracy: 0.3993 - grow_outputs_accuracy: 0.2892 - val_loss: 2.6200 - val_diease_outputs_loss: 1.1255 - val_grow_outputs_loss: 1.4945 - val_diease_outputs_accuracy: 0.4387 - val_grow_outputs_accuracy: 0.3336
Epoch 2/10
268/268 [==============================] - 299s 1s/step - loss: 2.6533 - diease_outputs_loss: 1.1367 - grow_outputs_loss: 1.5166 - diease_outputs_accuracy: 0.4220 - grow_outputs_accuracy: 0.3185 - val_loss: 2.6061 - val_diease_outputs_loss: 1.1173 - val_grow_outputs_loss: 1.4888 - val_diease_outputs_accuracy: 0.4387 - val_grow_outputs_accuracy: 0.3336
Epoch 3/10
268/268 [==============================] - 299s 1s/step - loss: 2.6319 - diease_outputs_loss: 1.1271 - grow_outputs_loss: 1.5048 - diease_outputs_accuracy: 0.4339 - grow_outputs_accu

diease_outputs_accuracy,▁▅▆▇▇▇█▇██
diease_outputs_loss,█▄▃▃▂▂▂▂▁▁
epoch,▁▂▃▃▄▅▆▆▇█
grow_outputs_accuracy,▁▅▆▅▆▇▇▆██
grow_outputs_loss,█▅▄▄▃▂▂▂▁▁
loss,█▄▃▃▃▂▂▂▁▁
val_diease_outputs_accuracy,▁▁▁▁▁▁▁▁▁█
val_diease_outputs_loss,█▅▆▄▇▃▃▄▁▁
val_grow_outputs_accuracy,▂▂▂▂█▇██▂▁
val_grow_outputs_loss,█▇▇▅▆▄▃▂▁▂
val_loss,█▆▆▅▆▄▃▃▁▂


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 3xwmfswn with config:
wandb: 	activation: elu
wandb: 	batch_size: 50
wandb: 	dense: 1024
wandb: 	dropout: 0.5
wandb: 	learning_rate: 3e-05
wandb: 	optimizer: sgd
wandb: 	pretrain_net: resnet
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13437 validated image filenames.
Found 3360 validated image filenames.
Epoch 1/10
268/268 [==============================] - 307s 1s/step - loss: 3.4999 - diease_outputs_loss: 1.4932 - grow_outputs_loss: 2.0066 - diease_outputs_accuracy: 0.2656 - grow_outputs_accuracy: 0.1735 - val_loss: 2.6659 - val_diease_outputs_loss: 1.1331 - val_grow_outputs_loss: 1.5328 - val_diease_outputs_accuracy: 0.4387 - val_grow_outputs_accuracy: 0.2176
Epoch 2/10
268/268 [==============================] - 300s 1s/step - loss: 2.9253 - diease_outputs_loss: 1.2328 - grow_outputs_loss: 1.6925 - diease_outputs_accuracy: 0.3776 - grow_outputs_accuracy: 0.2635 - val_loss: 2.6387 - val_diease_outputs_loss: 1.1220 - val_grow_outputs_loss: 1.5167 - val_diease_outputs_accuracy: 0.4390 - val_grow_outputs_accuracy: 0.2625
Epoch 3/10
268/268 [==============================] - 300s 1s/step - loss: 2.8518 - diease_outputs_loss: 1.2068 - grow_outputs_loss: 1.6450 - diease_outputs_accuracy: 0.3789 - grow_outputs_accu

In [ ]:
#wandb.agent('natutesf', train)

wandb: Agent Starting Run: zosqg6hl with config:
wandb: 	activation: softmax
wandb: 	batch_size: 20
wandb: 	dense: 32
wandb: 	dropout: 0.5
wandb: 	learning_rate: 3e-05
wandb: 	optimizer: rmsprop
wandb: 	pretrain_net: resnet
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hkleee. Use `wandb login --relogin` to force relogin


Found 13437 validated image filenames.
Found 3360 validated image filenames.


2022-06-17 18:35:30.847553: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-17 18:35:30.848671: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-17 18:35:30.893248: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-17 18:35:30.893901: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-06-17 18:35:30.893956: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-17 18:35:30.983316: I tensorflow/stream_executor/platform/default/dso_loade

Epoch 1/10


2022-06-17 18:35:38.584660: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-06-17 18:35:39.207826: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


...

wandb: Agent Starting Run: o9vqfwlc with config:
wandb: 	activation: relu
wandb: 	batch_size: 20
wandb: 	dense: 1024
wandb: 	dropout: 0.5
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: adam
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13437 validated image filenames.
Found 3360 validated image filenames.
Epoch 1/10
671/671 [==============================] - 320s 464ms/step - loss: 1.9489 - diease_outputs_loss: 0.8469 - grow_outputs_loss: 1.1020 - diease_outputs_accuracy: 0.6475 - grow_outputs_accuracy: 0.5374 - val_loss: 1.2642 - val_diease_outputs_loss: 0.4850 - val_grow_outputs_loss: 0.7793 - val_diease_outputs_accuracy: 0.8202 - val_grow_outputs_accuracy: 0.6756
Epoch 2/10
671/671 [==============================] - 306s 457ms/step - loss: 1.4511 - diease_outputs_loss: 0.5775 - grow_outputs_loss: 0.8736 - diease_outputs_accuracy: 0.7765 - grow_outputs_accuracy: 0.6348 - val_loss: 1.1621 - val_diease_outputs_loss: 0.4223 - val_grow_outputs_loss: 0.7398 - val_diease_outputs_accuracy: 0.8509 - val_grow_outputs_accuracy: 0.6792
Epoch 3/10
671/671 [==============================] - 308s 459ms/step - loss: 1.3144 - diease_outputs_loss: 0.5024 - grow_outputs_loss: 0.8120 - diease_outputs_accuracy: 0.8051 - grow_out

diease_outputs_accuracy,▁▄▆▆▆▇▇▇▇█
diease_outputs_loss,█▅▃▃▃▂▂▂▁▁
epoch,▁▂▃▃▄▅▆▆▇█
grow_outputs_accuracy,▁▄▆▆▇▇▇▇▇█
grow_outputs_loss,█▅▃▃▂▂▂▁▁▁
loss,█▅▃▃▂▂▂▂▁▁
val_diease_outputs_accuracy,▁▄▅▅▆▇▇▇██
val_diease_outputs_loss,█▅▄▃▃▂▁▂▁▂
val_grow_outputs_accuracy,▁▁▃▅▅▆▆███
val_grow_outputs_loss,█▆▅▃▃▂▂▁▁▁
val_loss,█▆▄▃▃▂▂▁▁▁


wandb: Agent Starting Run: h3tniy57 with config:
wandb: 	activation: relu
wandb: 	batch_size: 20
wandb: 	dense: 32
wandb: 	dropout: 0.5
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13437 validated image filenames.
Found 3360 validated image filenames.
Epoch 1/10
671/671 [==============================] - 318s 462ms/step - loss: 2.1285 - diease_outputs_loss: 0.9271 - grow_outputs_loss: 1.2014 - diease_outputs_accuracy: 0.6005 - grow_outputs_accuracy: 0.4926 - val_loss: 1.4409 - val_diease_outputs_loss: 0.5708 - val_grow_outputs_loss: 0.8701 - val_diease_outputs_accuracy: 0.7878 - val_grow_outputs_accuracy: 0.6452
Epoch 2/10
671/671 [==============================] - 308s 458ms/step - loss: 1.5601 - diease_outputs_loss: 0.6329 - grow_outputs_loss: 0.9273 - diease_outputs_accuracy: 0.7478 - grow_outputs_accuracy: 0.6046 - val_loss: 1.3919 - val_diease_outputs_loss: 0.5211 - val_grow_outputs_loss: 0.8708 - val_diease_outputs_accuracy: 0.7952 - val_grow_outputs_accuracy: 0.6220
Epoch 3/10
671/671 [==============================] - 305s 454ms/step - loss: 1.4475 - diease_outputs_loss: 0.5799 - grow_outputs_loss: 0.8675 - diease_outputs_accuracy: 0.7699 - grow_out

In [ ]:
#wandb.agent('natutesf', train)

wandb: Agent Starting Run: jkckd3xb with config:
wandb: 	activation: selu
wandb: 	batch_size: 20
wandb: 	dense: 32
wandb: 	dropout: 0.2
wandb: 	learning_rate: 3e-05
wandb: 	optimizer: rmsprop
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hkleee. Use `wandb login --relogin` to force relogin


Found 13437 validated image filenames.
Found 3360 validated image filenames.


2022-06-17 22:13:09.847772: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-17 22:13:09.848918: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-17 22:13:09.895292: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-17 22:13:09.895940: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-06-17 22:13:09.895996: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-17 22:13:09.976835: I tensorflow/stream_executor/platform/default/dso_loade

Epoch 1/10


2022-06-17 22:13:26.443949: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-06-17 22:13:27.100518: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


...

wandb: Agent Starting Run: 4hj0xi5o with config:
wandb: 	activation: elu
wandb: 	batch_size: 50
wandb: 	dense: 1024
wandb: 	dropout: 0.5
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: adam
wandb: 	pretrain_net: resnet
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13437 validated image filenames.
Found 3360 validated image filenames.
Epoch 1/10
268/268 [==============================] - 296s 1s/step - loss: 2.8774 - diease_outputs_loss: 1.2433 - grow_outputs_loss: 1.6341 - diease_outputs_accuracy: 0.3803 - grow_outputs_accuracy: 0.2988 - val_loss: 2.5410 - val_diease_outputs_loss: 1.0954 - val_grow_outputs_loss: 1.4456 - val_diease_outputs_accuracy: 0.4399 - val_grow_outputs_accuracy: 0.3369
Epoch 2/10
268/268 [==============================] - 293s 1s/step - loss: 2.5457 - diease_outputs_loss: 1.1054 - grow_outputs_loss: 1.4403 - diease_outputs_accuracy: 0.4408 - grow_outputs_accuracy: 0.3624 - val_loss: 2.4444 - val_diease_outputs_loss: 1.0776 - val_grow_outputs_loss: 1.3668 - val_diease_outputs_accuracy: 0.4589 - val_grow_outputs_accuracy: 0.4491
Epoch 3/10
268/268 [==============================] - 295s 1s/step - loss: 2.4844 - diease_outputs_loss: 1.0854 - grow_outputs_loss: 1.3989 - diease_outputs_accuracy: 0.4432 - grow_outputs_accu

diease_outputs_accuracy,▁▄▅▆▆▇█▇██
diease_outputs_loss,█▄▃▃▂▂▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
grow_outputs_accuracy,▁▄▅▆▇▇▇███
grow_outputs_loss,█▅▃▃▂▂▂▁▁▁
loss,█▅▃▃▂▂▂▁▁▁
val_diease_outputs_accuracy,▁▃▆▅█▁▇▇█▇
val_diease_outputs_loss,█▇▄▃▂▆▂▂▁▁
val_grow_outputs_accuracy,▁▆▇▅██▆███
val_grow_outputs_loss,█▅▄▃▃▂▃▁▁▁
val_loss,█▆▄▃▂▃▃▁▁▁


wandb: Agent Starting Run: pin2f72t with config:
wandb: 	activation: elu
wandb: 	batch_size: 20
wandb: 	dense: 1024
wandb: 	dropout: 0.5
wandb: 	learning_rate: 0.01
wandb: 	optimizer: rmsprop
wandb: 	pretrain_net: resnet
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13437 validated image filenames.
Found 3360 validated image filenames.
Epoch 1/10
671/671 [==============================] - 278s 410ms/step - loss: 20.2509 - diease_outputs_loss: 8.8778 - grow_outputs_loss: 11.3732 - diease_outputs_accuracy: 0.3411 - grow_outputs_accuracy: 0.2348 - val_loss: 9.6806 - val_diease_outputs_loss: 4.3298 - val_grow_outputs_loss: 5.3508 - val_diease_outputs_accuracy: 0.2789 - val_grow_outputs_accuracy: 0.1917
Epoch 2/10
671/671 [==============================] - 271s 404ms/step - loss: 12.1166 - diease_outputs_loss: 5.5022 - grow_outputs_loss: 6.6144 - diease_outputs_accuracy: 0.3433 - grow_outputs_accuracy: 0.2615 - val_loss: 14.5523 - val_diease_outputs_loss: 5.8892 - val_grow_outputs_loss: 8.6632 - val_diease_outputs_accuracy: 0.2696 - val_grow_outputs_accuracy: 0.2557
Epoch 3/10
671/671 [==============================] - 272s 405ms/step - loss: 12.1783 - diease_outputs_loss: 5.4795 - grow_outputs_loss: 6.6988 - diease_outputs_accuracy: 0.3548 - gro

In [ ]:
#wandb.agent('natutesf', train)

wandb: Agent Starting Run: 2cdut50y with config:
wandb: 	activation: selu
wandb: 	batch_size: 20
wandb: 	dense: 1024
wandb: 	dropout: 0.5
wandb: 	learning_rate: 0.01
wandb: 	optimizer: rmsprop
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hkleee. Use `wandb login --relogin` to force relogin


Found 13437 validated image filenames.
Found 3360 validated image filenames.


2022-06-18 02:03:43.231999: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-18 02:03:43.233210: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-18 02:03:43.278758: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 02:03:43.279434: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-06-18 02:03:43.279481: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-18 02:03:43.360683: I tensorflow/stream_executor/platform/default/dso_loade

Epoch 1/10


2022-06-18 02:04:00.327750: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-06-18 02:04:00.952906: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


...

wandb: Agent Starting Run: 1mmww7fq with config:
wandb: 	activation: elu
wandb: 	batch_size: 20
wandb: 	dense: 1024
wandb: 	dropout: 0.2
wandb: 	learning_rate: 0.001
wandb: 	optimizer: rmsprop
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13437 validated image filenames.
Found 3360 validated image filenames.
Epoch 1/10
671/671 [==============================] - 369s 535ms/step - loss: 3.2210 - diease_outputs_loss: 1.4421 - grow_outputs_loss: 1.7789 - diease_outputs_accuracy: 0.6316 - grow_outputs_accuracy: 0.5207 - val_loss: 1.1798 - val_diease_outputs_loss: 0.3786 - val_grow_outputs_loss: 0.8012 - val_diease_outputs_accuracy: 0.8568 - val_grow_outputs_accuracy: 0.6589
Epoch 2/10
671/671 [==============================] - 361s 538ms/step - loss: 1.3920 - diease_outputs_loss: 0.5091 - grow_outputs_loss: 0.8829 - diease_outputs_accuracy: 0.8030 - grow_outputs_accuracy: 0.6430 - val_loss: 1.2485 - val_diease_outputs_loss: 0.4286 - val_grow_outputs_loss: 0.8199 - val_diease_outputs_accuracy: 0.8485 - val_grow_outputs_accuracy: 0.6521
Epoch 3/10
671/671 [==============================] - 363s 541ms/step - loss: 1.2366 - diease_outputs_loss: 0.4200 - grow_outputs_loss: 0.8166 - diease_outputs_accuracy: 0.8446 - grow_out

diease_outputs_accuracy,▁▅▆▇▇▇▇███
diease_outputs_loss,█▄▃▂▂▂▂▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
grow_outputs_accuracy,▁▄▅▆▆▇▇███
grow_outputs_loss,█▄▃▃▂▂▂▁▁▁
loss,█▄▃▂▂▂▂▁▁▁
val_diease_outputs_accuracy,▃▂▅▁▆▇▆▇██
val_diease_outputs_loss,▅▇▃█▃▂▃▁▁▁
val_grow_outputs_accuracy,▁▁▃▄▃▇▇▇█▄
val_grow_outputs_loss,██▇▄▆▂▂▂▁▅
val_loss,▇█▆▇▅▂▂▂▁▄


wandb: Agent Starting Run: x17d6o5k with config:
wandb: 	activation: relu
wandb: 	batch_size: 50
wandb: 	dense: 1024
wandb: 	dropout: 0.2
wandb: 	learning_rate: 0.001
wandb: 	optimizer: rmsprop
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13437 validated image filenames.
Found 3360 validated image filenames.
Epoch 1/10
268/268 [==============================] - 331s 1s/step - loss: 2.6244 - diease_outputs_loss: 1.1737 - grow_outputs_loss: 1.4506 - diease_outputs_accuracy: 0.6410 - grow_outputs_accuracy: 0.5252 - val_loss: 1.2366 - val_diease_outputs_loss: 0.4511 - val_grow_outputs_loss: 0.7856 - val_diease_outputs_accuracy: 0.8339 - val_grow_outputs_accuracy: 0.6524
Epoch 2/10
268/268 [==============================] - 317s 1s/step - loss: 1.2553 - diease_outputs_loss: 0.4529 - grow_outputs_loss: 0.8025 - diease_outputs_accuracy: 0.8252 - grow_outputs_accuracy: 0.6506 - val_loss: 1.1090 - val_diease_outputs_loss: 0.4109 - val_grow_outputs_loss: 0.6982 - val_diease_outputs_accuracy: 0.8360 - val_grow_outputs_accuracy: 0.6920
Epoch 3/10
268/268 [==============================] - 319s 1s/step - loss: 1.1238 - diease_outputs_loss: 0.3974 - grow_outputs_loss: 0.7264 - diease_outputs_accuracy: 0.8421 - grow_outputs_accu

In [ ]:
#wandb.agent('natutesf', train)

wandb: Agent Starting Run: 76pv49f3 with config:
wandb: 	activation: selu
wandb: 	batch_size: 20
wandb: 	dense: 256
wandb: 	dropout: 0.2
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hkleee. Use `wandb login --relogin` to force relogin


Found 13437 validated image filenames.
Found 3360 validated image filenames.


2022-06-18 17:23:51.123500: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-18 17:23:51.124672: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-18 17:23:51.171422: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 17:23:51.172071: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-06-18 17:23:51.172115: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-18 17:23:51.254355: I tensorflow/stream_executor/platform/default/dso_loade

Epoch 1/10


2022-06-18 17:24:07.934218: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-06-18 17:24:08.563391: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


671/671 [==============================] - 332s 477ms/step - loss: 1.9475 - diease_outputs_loss: 0.8307 - grow_outputs_loss: 1.1168 - diease_outputs_accuracy: 0.6697 - grow_outputs_accuracy: 0.5429 - val_loss: 1.2931 - val_diease_outputs_loss: 0.4649 - val_grow_outputs_loss: 0.8282 - val_diease_outputs_accuracy: 0.8253 - val_grow_outputs_accuracy: 0.6592
Epoch 2/10
671/671 [==============================] - 311s 463ms/step - loss: 1.3464 - diease_outputs_loss: 0.5133 - grow_outputs_loss: 0.8331 - diease_outputs_accuracy: 0.7990 - grow_outputs_accuracy: 0.6485 - val_loss: 1.2183 - val_diease_outputs_loss: 0.4157 - val_grow_outputs_loss: 0.8027 - val_diease_outputs_accuracy: 0.8381 - val_grow_outputs_accuracy: 0.6601
Epoch 3/10
671/671 [==============================] - 310s 462ms/step - loss: 1.2412 - diease_outputs_loss: 0.4515 - grow_outputs_loss: 0.7897 - diease_outputs_accuracy: 0.8246 - grow_outputs_accuracy: 0.6694 - val_loss: 1.0847 - val_diease_outputs_loss: 0.3589 - val_grow_ou

diease_outputs_accuracy,▁▅▆▆▇▇████
diease_outputs_loss,█▄▃▃▂▂▂▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
grow_outputs_accuracy,▁▅▆▇▇▇████
grow_outputs_loss,█▄▄▃▂▂▁▁▁▁
loss,█▄▃▃▂▂▁▁▁▁
val_diease_outputs_accuracy,▁▃▆▅▆▇█▇██
val_diease_outputs_loss,█▆▃▃▃▂▁▂▁▁
val_grow_outputs_accuracy,▁▁▅▄▆▆▅█▆▆
val_grow_outputs_loss,█▇▃▄▂▃▃▁▁▂
val_loss,█▆▃▃▂▂▂▁▁▂


wandb: Agent Starting Run: izly5ftl with config:
wandb: 	activation: relu
wandb: 	batch_size: 20
wandb: 	dense: 256
wandb: 	dropout: 0.5
wandb: 	learning_rate: 0.001
wandb: 	optimizer: adam
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13437 validated image filenames.
Found 3360 validated image filenames.
Epoch 1/10
671/671 [==============================] - 316s 459ms/step - loss: 1.7791 - diease_outputs_loss: 0.7459 - grow_outputs_loss: 1.0331 - diease_outputs_accuracy: 0.6976 - grow_outputs_accuracy: 0.5623 - val_loss: 1.1558 - val_diease_outputs_loss: 0.3987 - val_grow_outputs_loss: 0.7571 - val_diease_outputs_accuracy: 0.8432 - val_grow_outputs_accuracy: 0.6750
Epoch 2/10
671/671 [==============================] - 305s 454ms/step - loss: 1.2936 - diease_outputs_loss: 0.5030 - grow_outputs_loss: 0.7906 - diease_outputs_accuracy: 0.8014 - grow_outputs_accuracy: 0.6571 - val_loss: 1.0708 - val_diease_outputs_loss: 0.3653 - val_grow_outputs_loss: 0.7055 - val_diease_outputs_accuracy: 0.8509 - val_grow_outputs_accuracy: 0.6926
Epoch 3/10
335/671 [=============>................] - ETA: 2:14 - loss: 1.1658 - diease_outputs_loss: 0.4179 - grow_outputs_loss: 0.7480 - diease_outputs_accuracy: 0.8327 - grow_outputs_a

In [ ]:
#wandb.agent('natutesf', train)

wandb: Agent Starting Run: s8pzh9tx with config:
wandb: 	activation: elu
wandb: 	batch_size: 20
wandb: 	dense: 32
wandb: 	dropout: 0.2
wandb: 	learning_rate: 0.001
wandb: 	optimizer: adam
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hkleee. Use `wandb login --relogin` to force relogin


Found 13437 validated image filenames.
Found 3360 validated image filenames.


2022-06-18 20:49:59.282088: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-18 20:49:59.283242: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-18 20:49:59.330269: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 20:49:59.330925: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-06-18 20:49:59.330969: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-18 20:49:59.408766: I tensorflow/stream_executor/platform/default/dso_loade

Epoch 1/10


2022-06-18 20:50:17.031704: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-06-18 20:50:17.683322: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


671/671 [==============================] - 347s 499ms/step - loss: 1.7026 - diease_outputs_loss: 0.7120 - grow_outputs_loss: 0.9906 - diease_outputs_accuracy: 0.7126 - grow_outputs_accuracy: 0.5807 - val_loss: 1.2099 - val_diease_outputs_loss: 0.4677 - val_grow_outputs_loss: 0.7421 - val_diease_outputs_accuracy: 0.8074 - val_grow_outputs_accuracy: 0.6807
Epoch 2/10
671/671 [==============================] - 320s 476ms/step - loss: 1.1966 - diease_outputs_loss: 0.4448 - grow_outputs_loss: 0.7518 - diease_outputs_accuracy: 0.8254 - grow_outputs_accuracy: 0.6827 - val_loss: 1.0472 - val_diease_outputs_loss: 0.3547 - val_grow_outputs_loss: 0.6925 - val_diease_outputs_accuracy: 0.8577 - val_grow_outputs_accuracy: 0.6938
Epoch 3/10
671/671 [==============================] - 323s 481ms/step - loss: 1.0984 - diease_outputs_loss: 0.3970 - grow_outputs_loss: 0.7013 - diease_outputs_accuracy: 0.8524 - grow_outputs_accuracy: 0.7068 - val_loss: 0.9822 - val_diease_outputs_loss: 0.3431 - val_grow_ou

diease_outputs_accuracy,▁▅▆▆▇▇▇▇██
diease_outputs_loss,█▄▃▃▃▂▂▂▁▁
epoch,▁▂▃▃▄▅▆▆▇█
grow_outputs_accuracy,▁▄▅▆▆▇▇▇▇█
grow_outputs_loss,█▅▄▃▃▂▂▂▁▁
loss,█▄▄▃▃▂▂▂▁▁
val_diease_outputs_accuracy,▁▄▅▆▅▇▇█▇█
val_diease_outputs_loss,█▅▄▃▄▂▂▁▂▁
val_grow_outputs_accuracy,▁▂▅▆▄▄█▆██
val_grow_outputs_loss,█▆▄▃▃▄▁▂▁▁
val_loss,█▅▄▃▄▃▁▂▁▁


wandb: Agent Starting Run: 3qxqsocj with config:
wandb: 	activation: relu
wandb: 	batch_size: 50
wandb: 	dense: 1024
wandb: 	dropout: 0.2
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13437 validated image filenames.
Found 3360 validated image filenames.
Epoch 1/10
268/268 [==============================] - 350s 1s/step - loss: 6.3235 - diease_outputs_loss: 2.3241 - grow_outputs_loss: 3.9994 - diease_outputs_accuracy: 0.6572 - grow_outputs_accuracy: 0.5261 - val_loss: 1.0909 - val_diease_outputs_loss: 0.3622 - val_grow_outputs_loss: 0.7287 - val_diease_outputs_accuracy: 0.8655 - val_grow_outputs_accuracy: 0.6801
Epoch 2/10
268/268 [==============================] - 338s 1s/step - loss: 1.2096 - diease_outputs_loss: 0.4305 - grow_outputs_loss: 0.7791 - diease_outputs_accuracy: 0.8350 - grow_outputs_accuracy: 0.6702 - val_loss: 1.1584 - val_diease_outputs_loss: 0.4152 - val_grow_outputs_loss: 0.7431 - val_diease_outputs_accuracy: 0.8432 - val_grow_outputs_accuracy: 0.6815
Epoch 3/10
189/268 [====================>.........] - ETA: 1:28 - loss: 1.1259 - diease_outputs_loss: 0.3902 - grow_outputs_loss: 0.7357 - diease_outputs_accuracy: 0.8523 - grow_outputs_accurac

In [ ]:
wandb.agent('natutesf', train)

wandb: Agent Starting Run: d2zycd95 with config:
wandb: 	activation: elu
wandb: 	batch_size: 20
wandb: 	dense: 256
wandb: 	dropout: 0.5
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: rmsprop
wandb: 	pretrain_net: resnet
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hkleee. Use `wandb login --relogin` to force relogin


Found 13437 validated image filenames.
Found 3360 validated image filenames.


2022-06-18 22:04:49.476294: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-18 22:04:49.477431: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-18 22:04:49.521924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 22:04:49.522542: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-06-18 22:04:49.522590: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-18 22:04:49.600824: I tensorflow/stream_executor/platform/default/dso_loade

Epoch 1/10


2022-06-18 22:04:57.030035: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-06-18 22:04:57.644610: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


...

wandb: Agent Starting Run: uwq1766q with config:
wandb: 	activation: selu
wandb: 	batch_size: 20
wandb: 	dense: 1024
wandb: 	dropout: 0.5
wandb: 	learning_rate: 0.001
wandb: 	optimizer: adam
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13437 validated image filenames.
Found 3360 validated image filenames.
Epoch 1/10
671/671 [==============================] - 320s 465ms/step - loss: 2.4553 - diease_outputs_loss: 1.0244 - grow_outputs_loss: 1.4309 - diease_outputs_accuracy: 0.6804 - grow_outputs_accuracy: 0.5491 - val_loss: 1.2446 - val_diease_outputs_loss: 0.4912 - val_grow_outputs_loss: 0.7534 - val_diease_outputs_accuracy: 0.8086 - val_grow_outputs_accuracy: 0.6747
Epoch 2/10
671/671 [==============================] - 311s 463ms/step - loss: 1.4702 - diease_outputs_loss: 0.5842 - grow_outputs_loss: 0.8860 - diease_outputs_accuracy: 0.7726 - grow_outputs_accuracy: 0.6316 - val_loss: 1.2840 - val_diease_outputs_loss: 0.4591 - val_grow_outputs_loss: 0.8249 - val_diease_outputs_accuracy: 0.8262 - val_grow_outputs_accuracy: 0.6664
Epoch 3/10
671/671 [==============================] - 311s 463ms/step - loss: 1.3923 - diease_outputs_loss: 0.5406 - grow_outputs_loss: 0.8517 - diease_outputs_accuracy: 0.7899 - grow_out

diease_outputs_accuracy,▁▄▅▆▇▇▇███
diease_outputs_loss,█▄▃▂▂▂▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
grow_outputs_accuracy,▁▄▅▆▆▇▇███
grow_outputs_loss,█▃▃▂▂▂▁▁▁▁
loss,█▄▃▂▂▂▁▁▁▁
val_diease_outputs_accuracy,▁▂▄▇▆▆▆█▇▄
val_diease_outputs_loss,█▇▅▂▂▂▃▁▃▆
val_grow_outputs_accuracy,▃▃▆▆▁▇█▆▅█
val_grow_outputs_loss,▄▆▂▂█▁▁▃▅▁
val_loss,▇█▄▂▇▁▂▂▄▃


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: wggn7g4j with config:
wandb: 	activation: elu
wandb: 	batch_size: 50
wandb: 	dense: 256
wandb: 	dropout: 0.2
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: rmsprop
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13437 validated image filenames.
Found 3360 validated image filenames.
Epoch 1/10
268/268 [==============================] - 333s 1s/step - loss: 2.0327 - diease_outputs_loss: 0.8806 - grow_outputs_loss: 1.1521 - diease_outputs_accuracy: 0.6265 - grow_outputs_accuracy: 0.5063 - val_loss: 1.3729 - val_diease_outputs_loss: 0.5382 - val_grow_outputs_loss: 0.8346 - val_diease_outputs_accuracy: 0.8015 - val_grow_outputs_accuracy: 0.6393
Epoch 2/10
268/268 [==============================] - 321s 1s/step - loss: 1.4167 - diease_outputs_loss: 0.5709 - grow_outputs_loss: 0.8458 - diease_outputs_accuracy: 0.7837 - grow_outputs_accuracy: 0.6491 - val_loss: 1.2412 - val_diease_outputs_loss: 0.4477 - val_grow_outputs_loss: 0.7935 - val_diease_outputs_accuracy: 0.8396 - val_grow_outputs_accuracy: 0.6613
Epoch 3/10
268/268 [==============================] - 319s 1s/step - loss: 1.2888 - diease_outputs_loss: 0.4884 - grow_outputs_loss: 0.8004 - diease_outputs_accuracy: 0.8144 - grow_outputs_accu